In [1]:
import ee
import ee.batch
import pyperclip

In [2]:
# Initialize the Earth Engine library.
ee.Authenticate()
ee.Initialize(project='pixelcampione-satelital2map')


In [3]:
import ee.data


satellital_ids = []
elevation_ids = []

In [4]:
import ee.batch


def export_elevation(dataset_image,area_of_interest, filename, copy: bool = True):
    dataset = ee.image.Image(dataset_image)

    selected = dataset.select('elevation')
    # Define export parameters.
    task = ee.batch.Export.image.toDrive(**{
        'image': selected,
        'description': 'elevation_to_drive',
        'fileNamePrefix': filename,
        'scale': 90,  # Adjust scale if necessary
        'region': area_of_interest,
        'fileFormat': 'GeoTIFF',
        'folder': 'dataset'  # Specify the Google Drive folder name here.
    })

    # Start the export task.
    task.start()

    # Print the task ID
    print(f'Task ID: {task.id}')
    elevation_ids.append(task.id)


def export_rgb(image: ee.image.Image,area_of_interest, filename, copy: bool = True):
        
    # Select the RGB bands
    selected = image.select(['B4', 'B3', 'B2'])

    # Define export parameters.
    task = ee.batch.Export.image.toDrive(**{
        'image': selected,
        'description':'rgb_to_drive',
        'fileNamePrefix': filename,
        'scale': 30,  # Adjust scale if necessary
        'region': area_of_interest,
        'fileFormat': 'GeoTIFF',
        'folder': 'dataset'  # Specify the Google Drive folder name here.
    })

    # Start the export task.
    task.start()

    # Print the task ID
    print(f'Task ID: {task.id}')
    satellital_ids.append(task.id)

In [9]:
center = {
    'longitude': -120.0,
    'latitude': 40.0
}
offset = 0.10

# 5 x 5
lat_mult = 2
lon_mult = 0
index = {
    'start_lat': 6*lat_mult,
    'start_lon': 6*lon_mult,
    'end_lat': 6*(lat_mult+1),
    'end_lon': 6*(lon_mult+1)
}
# Me quede en 145
temp = 361

In [10]:
counter = temp
for i in range(index['start_lat'],index['end_lat']):
    for j in range(index['start_lon'], index['end_lon']):
        # Define an area of interest (for example, a bounding box around Mt. Everest).
        start = ee.geometry.Geometry.Rectangle([center['longitude'] - offset + 2*offset*j, center['latitude'] - offset - 2*offset*i
                                                , center['longitude'] + offset+ 2*offset*j, center['latitude'] + offset - 2*offset*i])
        # Call the function with your area of interest and desired filename prefix.
        export_elevation("CGIAR/SRTM90_V4",start, f'elevation_{counter}')
        counter += 1

        print("j = ",j)
    print("i = ",i)

print("-------------------------ELEVATION FINISHED-------------------------")

Task ID: E3Z2JLXGCFZQXGTCJRCNCOOR
j =  24
Task ID: NYAC6DCY5OYETRICQYIS6IWT
j =  25
Task ID: S6KMLDNKRVH2KTXSURJ6H4PX
j =  26
Task ID: 7BEOQ4UG5YKVBSW3QCNAD4DA
j =  27
Task ID: 6HTAAZ6FG2GDJENOSMPVBBNV
j =  28
Task ID: 6PQ6TEJHWUMBKZHOCTEONHJI
j =  29
i =  6
Task ID: XLNODXX3ZEKJ7BY72QF2YUDN
j =  24
Task ID: FJCUEJWAN5LL2QY2MOVEEMR2
j =  25
Task ID: V7YIOGGXFMFTAL7PCU4YZENO
j =  26
Task ID: 6ZU4BZKUI7S6QZBH3AYCLGAS
j =  27
Task ID: 3H43MH3AGOJB5QKJPPAM3HWP
j =  28
Task ID: EX6FGC6CGC3ZMWZYIUFIINNU
j =  29
i =  7
Task ID: ZYB7R534MYWGOREA5ERHWSYU
j =  24
Task ID: USVOPL4AZFEXJ4A2KRHNXM74
j =  25
Task ID: TLCIXKDIJ3ORNXHFAWLJ46IX
j =  26
Task ID: 6S3HJGQBMXVJZVMKZ262WGQW
j =  27
Task ID: QVFQNFDX5V2AVOBJYYBHDPFI
j =  28
Task ID: FH4JPP25HZXT7RXNYDLHTZEP
j =  29
i =  8
Task ID: CXKEDAKDAJGMODE5DYKRKTVN
j =  24
Task ID: D7I2FIBQIHWRKKQO4QF44PTH
j =  25
Task ID: 25PPOGM4KKHOIT6YUEY55XCJ
j =  26
Task ID: R36VWYZRXCO457AGRRGB7NLM
j =  27
Task ID: P52XP3GP2UJB53W7DYF2I45T
j =  28
Task ID: N6F5

In [11]:
counter = temp
for i in range(index['start_lat'],index['end_lat']):
    for j in range(index['start_lon'], index['end_lon']):
        # Define an area of interest (for example, a bounding box around Mt. Everest).
        start = ee.geometry.Geometry.Rectangle([center['longitude'] - offset + 2*offset*j, center['latitude'] - offset + 2*offset*i
                                                , center['longitude'] + offset+ 2*offset*j, center['latitude'] + offset + 2*offset*i])

        # Convert ImageCollection to Image
        image = ee.imagecollection.ImageCollection("LANDSAT/LC09/C02/T1").filterBounds(start).mosaic()
        
        # Call the function with your area of interest and desired filename prefix.
        export_rgb(image, start, f'satellite_{counter}')
        counter += 1
        

        print("j = ",j)
    print("i = ",i)

print("-------------------------SATELLITE FINISHED-------------------------")

Task ID: YBOAYH2FDU2QKSOF6FNOENA3
j =  24
Task ID: FNY2OZ5D5IFW5E2NPXHG3U4L
j =  25
Task ID: AYLUIXVUN2DRQQT3JDHENA5U
j =  26
Task ID: D65PMS22G3TTP36LLOHHJ54J
j =  27
Task ID: RLTR3B26CVOJ75Q6WOPPO47O
j =  28
Task ID: MBMG4DG43DDV6FVZLICE4BBT
j =  29
i =  6
Task ID: VYKUXPTZ3G64MZMUE23GIYEU
j =  24
Task ID: 62I5XT6QD4MWC4EO2NCT3A32
j =  25
Task ID: P4X6SM2HSVACDA7IQJNOTGLJ
j =  26
Task ID: WAO5A4HYINZYTHG3NUZFZHSZ
j =  27
Task ID: GOSISWQNKAG52TT3C6KXXB5X
j =  28
Task ID: N7TER74GLJMOYKOPTLPIDBWO
j =  29
i =  7
Task ID: 2ZRZVJGEYOI6YABLBEALHFZ2
j =  24
Task ID: OJUUYNNE6EUNEWN6AAAZ7LO6
j =  25
Task ID: TDPSPEPV7KZC2H5FUARFGI5H
j =  26
Task ID: M53DWHHPYML2SDZTBR2BLOZR
j =  27
Task ID: OYLOLS3UDAXITICKL6YQGOVY
j =  28
Task ID: FIPQPKJ3KQKGLWGU6S3JBZRV
j =  29
i =  8
Task ID: YJSZN2755WQ634R4CPAOHRRK
j =  24
Task ID: 76QRYSU76I67D4OBHCLIEFQM
j =  25
Task ID: NLVNE7ODNYVC2PLI46GMFPEY
j =  26
Task ID: Z5RMKY2DOJUJOFDT576KULRH
j =  27
Task ID: 6WZB5R3Y6AZ6SRWE4DLDHX2Z
j =  28
Task ID: FSWG

In [12]:
print("Export finished!")
pyperclip.copy("\'satellital\': " + str(satellital_ids)+"," +"\n"
               "\'elevation\': " + str(elevation_ids) + "\n")


Export finished!
